Doing the prediction on loading the model


In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle


In [20]:
# load the model / scaler / labelencoder/OHE
model = load_model('churn_model.h5')
# Load the scaler and label encoder
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load the OneHotEncoder
with open('ohe.pkl', 'rb') as f:
    ohe = pickle.load(f)

    


In [21]:
test_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 90000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 50000.0
}

# Convert the test data to a DataFrame
test_df = pd.DataFrame([test_data])
test_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,35,5,90000.0,2,1,0,50000.0


 Syntax	Type	Shape	Use Case
df['Geography']	Series	1D: (n_samples,)	When you want a single column as a vector
df[['Geography']]	DataFrame	2D: (n_samples, 1)	Required by transformers like OneHotEncoder


In [ ]:
geo = ohe.transform(test_df[['Geography']]).toarray()
geo_df = pd.DataFrame(geo, columns=ohe.get_feature_names_out(['Geography']))
test_df = pd.concat([test_df, geo_df], axis=1)
test_df.drop(columns=['Geography'], inplace=True)
test_df

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [23]:
test_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,Male,35,5,90000.0,2,1,0,50000.0,1.0,0.0,0.0


In [24]:
# LAbel encoder
test_df['Gender']= label_encoder.transform(test_df['Gender'])



c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [25]:
test_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,35,5,90000.0,2,1,0,50000.0,1.0,0.0,0.0


In [26]:
test_scaled = scaler.transform(test_df)

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [27]:
test_scaled

array([[-0.01709861,  0.91324755, -0.37056859, -0.00134472,  0.2225185 ,
         0.80843615,  0.64920267, -1.02583358, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [29]:
#Predict the churn
predictions = model.predict(test_scaled)
# Convert predictions to binary (0 or 1)
predictions


1/1 [==============================] - 0s 29ms/step


array([[0.02844858]], dtype=float32)

In [30]:
#Prediction binary
predictions_binary = (predictions > 0.5).astype(int)
predictions_binary

array([[0]])